# Create MBTiles for Soils-revealed platform

Check this [notebook](https://github.com/Vizzuality/sci_team_data_bank/blob/master/Encyclopedia/map_tile_processing/MBTiles_from_Carto_data.ipynb) for further information on the creation of `MBTiles`.

## Table of Contents
### [Python libraries](#libraries)
### [Utils](#utils)
- **[df_from_carto](#df_from_carto)**
- **[long_lasting_SQL_queries](#long_lasting_SQL_queries)**
- **[create_mbtiles](#create_mbtiles)**

### [Read data from different sources ](#read_data)
- **[Biomes](#biomes)**
- **[World Database on Protected Areas](#protected_areas)**
- **[River basins](#river_basins)**
- **[Political boundaries](#political_boundaries)**

### [Create `MBTiles`](#create_mbtiles_2)
### [Show `MBTiles` in our localhost](#show_mbtiles)

<a id='libraries'></a>
### Python libraries

In [1]:
import os
import requests
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely.wkb 
from shapely.ops import cascaded_union
from carto.auth import APIKeyAuthClient
from carto.sql import BatchSQLClient
from carto.sql import SQLClient
from tqdm import tqdm
import getpass
import subprocess
import time
import LMIPy

<a id='utils'></a>
### Utils
<a id='df_from_carto'></a>
**df_from_carto**

In [2]:
def df_from_carto(account, query):
    """
    It gets data by querying a carto table and converts it into a GeoDataFrame.
    """
    urlCarto = f"https://{account}.carto.com/api/v2/sql"
    
    sql = {"q": query}
    r = requests.get(urlCarto, params=sql)
    
    data = r.json()
    
    df = gpd.GeoDataFrame(data.get("rows"))
    if 'the_geom' in df.columns:
        # Change geometry from WKB to WKT format
        df['geometry'] = df.apply(lambda x: shapely.wkb.loads(x['the_geom'],hex=True), axis=1 )
        df.drop(columns=['the_geom'], inplace=True)
        if 'the_geom_webmercator' in df.columns:
            df.drop(columns=['the_geom_webmercator'], inplace=True)
        df.crs = {'init': 'epsg:4326'}
        df = df.to_crs({'init': 'epsg:4326'})
        
    return df

<a id='long_lasting_SQL_queries'></a>
**long_lasting_SQL_queries**

In [3]:
def long_lasting_SQL_queries(account, query, api_key):
    # For long lasting SQL queries we use the batch SQL API.
    table_name = 'job_result'
    
    base_url = f'https://{account}.carto.com/'
    auth_client = APIKeyAuthClient(api_key=api_key, base_url=base_url)
    
    sql_query =(f'SELECT * INTO {table_name} FROM ({query}) as job')
    
    LIST_OF_SQL_QUERIES = [sql_query]
    
    batchSQLClient = BatchSQLClient(auth_client)
    createJob = batchSQLClient.create(LIST_OF_SQL_QUERIES)
    
    # Check the status of a job with the job_id every 10 s
    readJob = batchSQLClient.read(createJob['job_id'])
    
    timeout = time.time() + 60*60  # 1 hour from now
    while readJob.get('status') != 'done':
        time.sleep(10)
        print(readJob.get('status'))
        if readJob.get('status') == 'failed':
            print('Job failed.')
            break
        if time.time() > timeout:
            readJob = batchSQLClient.read(createJob['job_id'])
            # Cancel a job given its job_id
            if readJob.get('status') != 'donne':
                cancelJob = batchSQLClient.cancel(createJob['job_id'])     
                print('Job cancelled after 1 hour running')
                break
            
        readJob = batchSQLClient.read(createJob['job_id'])
       
    # Read the table
    sql = SQLClient(auth_client)
    data = sql.send("select * from "+table_name)
    
    # Drop the table
    sql = SQLClient(auth_client)
    sql.send("DROP TABLE "+table_name)
    
    df = gpd.GeoDataFrame(data.get("rows"))
    if 'the_geom' in df.columns:
        # Change geometry from WKB to WKT format
        df['geometry'] = df.apply(lambda x: shapely.wkb.loads(x['the_geom'],hex=True), axis=1 )
        df.drop(columns=['the_geom'], inplace=True)
        if 'the_geom_webmercator' in df.columns:
            df.drop(columns=['the_geom_webmercator'], inplace=True)
        df.crs = {'init': 'epsg:4326'}
        df = df.to_crs({'init': 'epsg:4326'})
        
    return df

**merge_geometries**

In [4]:
def merge_geometries(df, column_name):
    df_new = pd.DataFrame(columns=list(df.columns))
    geom = []
    for value in tqdm(df[column_name].unique()):
        df_tmp = df[df[column_name] == value].iloc[:1]
        geom.append(cascaded_union(list(df[df[column_name] == value].geometry)))
        
        df_new = pd.concat([df_new, df_tmp])
        
    df_new.reset_index(inplace=True)
    df_new.drop(columns='index', inplace=True)
    df_new.drop(columns='geometry', inplace=True)
    df_new['geometry'] = geom

    return gpd.GeoDataFrame(df_new)

<a id='create_mbtiles'></a>
**create_mbtiles**

In [5]:
def create_mbtiles(source_path, dest_path, layer_name, opts="-zg --drop-densest-as-needed --extend-zooms-if-still-dropping --force --read-parallel"):
    """
    Use tippecanoe to create a MBTILE at dest_path from source_path.
    layer_name is used for the name of the layer in the MBTILE.
    Regex file path (/*.geojson) is supported for source_path.
    """
    cmd = f"tippecanoe -o {dest_path} -l {layer_name} {opts} {source_path}"
    print(f"Processing: {cmd}")
    r = subprocess.call(cmd, shell=True)
    if r == 0:
        print("Task created")
    else:
        print("Task failed")
    print("Finished processing")

<a id='read_data'></a>
### Read data from different sources 

<a id='biomes'></a>
#### **[Biomes](https://resourcewatch.org/data/explore/bio042-Ecoregion-by-Biome)**

In [ ]:
ds = LMIPy.Dataset('ed1544bb-a092-424e-88c2-8d548f4ef94a')
ds

In [ ]:
biomes = gpd.read_file('../data/mbtiles/bio_042_ecoregions_by_biome_1_14/bio_042_ecoregions_by_biome_1_14.shp')
biomes.drop(columns='cartodb_id', inplace=True)
biomes.head()

**Merge geometries**

In [ ]:
biomes_0 = merge_geometries(biomes, 'biome_name')

In [ ]:
a = np.empty(len(biomes_0))
a[:] = np.nan
biomes_0['eco_name'] = a
biomes_0['realm'] = a
biomes_0['eco_biome_'] = a
biomes_0['eco_id'] = a

Append `GeoDataFrames`

In [ ]:
biomes_0['level'] = 0
biomes['level'] = 1

In [ ]:
data = biomes_0.append(biomes, sort=False)
data = data[['biome_name', 'biome_num', 'eco_name', 'realm', 'eco_biome_', 'eco_id',
       'level', 'geometry']]
data

**Add id**

In [ ]:
data['id'] = np.arange(len(data))

In [ ]:
data.head()

Save `GeoDataFrame` as `GeoJSON`

In [ ]:
data.to_file("../data/mbtiles/ecoregions_by_biome.json", driver="GeoJSON")

Save `GeoDataFrame` as `Shapefile`

In [ ]:
data.to_file("../data/mbtiles/ecoregions_by_biome/ecoregions_by_biome.shp")

<a id='protected_areas'></a>
#### **[World Database on Protected Areas](https://resourcewatch.org/data/explore/bio007-World-Database-on-Protected-Areas_replacement)**

In [ ]:
ds = LMIPy.Dataset('2442891a-157a-40e6-9092-ee596e6d30ba')
ds

In [ ]:
areas = gpd.read_file('../data/mbtiles/wdpa_protected_areas/wdpa_protected_areas.shp')
areas.drop(columns='cartodb_id', inplace=True)
areas.head()

In [ ]:
areas_filter = areas[(areas['marine'] == '0') & (areas['iucn_cat'] != 'Not Assigned') & (areas['iucn_cat'] != 'Not Applicable') & (areas['iucn_cat'] != 'Not Reported')]

In [ ]:
areas['iucn_cat'].unique()

In [ ]:
account = "wri-01"
api_key = getpass.getpass('Carto account api key:')

In [ ]:
query = 'SELECT distinct(iucn_cat) FROM wdpa_protected_areas'
df = df_from_carto(account, query)
iucn_cats = list(df['iucn_cat'])

In [ ]:
query = 'SELECT * FROM wdpa_protected_areas LIMIT 1'
df = df_from_carto(account, query)
coulmns = list(df.columns)

In [ ]:
data = gpd.GeoDataFrame(columns=coulmns)

for iucn_cat in iucn_cats:
    query = "SELECT * FROM wdpa_protected_areas WHERE iucn_cat = 'Ia' OR iucn_cat = 'Ib' OR iucn_cat = 'II' OR iucn_cat = 'III'"

    df = long_lasting_SQL_queries(account, query, api_key)
    
    data = pd.concat([data, df], ignore_index=True)

In [ ]:
data.head()

<a id='river_basins'></a>
#### **River basins** ([source](http://www.fao.org/nr/water/aquamaps/))
**Major hydrological basins**

In [ ]:
major = gpd.read_file('../data/mbtiles/Major_hydrological_basins/major_hydrobasins.shp')
major.columns = map(str.lower, major.columns)
major.head()

**Merge geometries**

In [ ]:
major = merge_geometries(major, 'maj_bas')

Save `GeoDataFrame` as `GeoJSON`

In [ ]:
major.to_file("../data/mbtiles/major_hydrological_basins.json", driver="GeoJSON")

**Minor hydrological basins**

In [ ]:
account = "wri-rw"
api_key = getpass.getpass('Carto account api key:')

In [ ]:
query = "SELECT maj_bas, maj_name, maj_area, sub_bas, sub_name, sub_area, the_geom FROM hydrobasins_fao_fiona_merged_v01"

minor = long_lasting_SQL_queries(account, query, api_key)

In [ ]:
minor.head()

**Merge geometries**

In [ ]:
minor = merge_geometries(minor, 'sub_bas')

Save `GeoDataFrame` as `GeoJSON`

In [ ]:
minor.to_file("../data/mbtiles/minor_hydrological_basins.json", driver="GeoJSON")

Append `GeoDataFrames`

In [ ]:
major['level'] = 0
minor['level'] = 1

In [ ]:
data = major.append(minor, sort=False)
data = data[['maj_bas', 'maj_name', 'maj_area', 'sub_bas', 'sub_name', 'sub_area',
       'level', 'geometry']]
data

**Add id**

In [ ]:
data['id'] = np.arange(len(data))

Save `GeoDataFrame` as `GeoJSON`

In [ ]:
data.to_file("../data/mbtiles/hydrological_basins.json", driver="GeoJSON")

Save `GeoDataFrame` as `Shapefile`

In [ ]:
data.to_file("../data/mbtiles/hydrological_basins/hydrological_basins.shp")

<a id='political_boundaries'></a>
#### **Political boundaries ([source](https://gadm.org/data.html))**

**gadm36 political boundaries**

In [ ]:
account = "wri-01"
api_key = getpass.getpass('Carto account api key:')

In [ ]:
account = "wri-01"

query = "SELECT name_0, name_1, name_2, area, size, level, gid_0, gid_1, gid_2, the_geom FROM gadm36_political_boundaries"

data = long_lasting_SQL_queries(account, query, api_key)
data.head()

In [ ]:
data.plot()

In [ ]:
gadm36 = gpd.read_file('../data/mbtiles/gadm36_political_boundaries/gadm36_political_boundaries.shp')
gadm36.drop(columns='cartodb_id', inplace=True)
gadm36.head()

In [ ]:
gadm36 = gadm36[gadm36['level'].isin([0,1])]

**Add id**

In [ ]:
gadm36['id'] = np.arange(len(gadm36))

Save `GeoDataFrame` as `GeoJSON`

In [ ]:
gadm36.to_file("../data/mbtiles/gadm36_political_boundaries.json", driver="GeoJSON")

Save `GeoDataFrame` as `Shapefile`

In [ ]:
gadm36.to_file("../data/mbtiles/political_boundaries/political_boundaries.shp")

**Disputed boundaries 2018**

In [ ]:
account = "wri-01"

query = "SELECT gid_0, name_0, name, note, the_geom FROM disputed_boundaries_2018 WHERE gid_0 in ('PAK', 'IND', 'CHN') AND name in ('Indian claim', 'Pakistani claim', 'Chinese claim')"

data = df_from_carto(account, query)
data

#### Major Physical Features ([source](file:///Users/ikersanchez/Vizzuality/GitHub/sci_team_data_bank/Projects/soils-revealed/data/mbtiles/ne_10m_geography_regions_elevation_points/ne_10m_geography_regions_elevation_points.README.html))
**Physical areas**

In [24]:
polys = gpd.read_file('../data/mbtiles/ne_10m_geography_regions_polys/ne_10m_geography_regions_polys.shp')
polys = polys[~polys.featurecla.isin(['Continent','Island group', 'Dragons-be-here', 'Lake'])]
polys.head()

,featurecla,name,namealt,region,subregion,min_label,max_label,scalerank,label,wikidataid,...,name_pl,name_pt,name_ru,name_sv,name_tr,name_vi,name_zh,wdid_score,ne_id,geometry
0,Island,Pentecost,None,Oceania,Melanesia,6.7,12.0,7,Pentecost I.,Q960808,...,Pentecost,Pentecostes,Пентекост,Pentecost Island,None,None,彭特科斯特岛,4,1159102327,"POLYGON ((168.22657 -15.98968, 168.20411 -15.9..."
1,Island,Anatom,None,Oceania,Melanesia,6.7,12.0,7,Anatom,Q528884,...,Anatom,Aneytium,Анейтьюм,None,Anatom,None,阿納托姆島,4,1159102329,"POLYGON ((169.83790 -20.24737, 169.82997 -20.2..."
2,Island,Gaua,None,Oceania,Melanesia,6.7,12.0,7,Gaua,Q583383,...,Gaua,Gaua,Гауа,None,None,None,加瓦島,4,1159102331,"POLYGON ((167.52491 -14.14214, 167.58619 -14.1..."
3,Island,Utupua,None,Oceania,Melanesia,6.7,12.0,7,Utupua I.,Q2081472,...,None,None,Утупуа,Utupua,None,None,None,4,1159102333,"POLYGON ((166.52223 -11.27068, 166.52711 -11.2..."
4,Island,Vanikolo,None,Oceania,Melanesia,6.7,12.0,7,Vanikolo,Q1536424,...,Vanikoro,None,Ваникоро,Vanikoro,None,None,None,4,1159102335,"POLYGON ((166.93263 -11.66021, 166.91822 -11.6..."


**Merge geometries**

In [25]:
polys_0 = merge_geometries(polys, 'featurecla')

100%|██████████| 19/19 [00:00<00:00, 40.08it/s]


In [26]:
a = np.empty(len(polys_0))
a[:] = np.nan
for column in list(polys_0.columns)[1:-1]:
    polys_0[column] = a

Append `GeoDataFrames`

In [27]:
polys_0['level'] = 0
polys['level'] = 1

data = polys_0.append(polys, sort=False)

data = data[['featurecla', 'name', 'region', 'subregion', 'ne_id', 'level', 'geometry']]
data

,featurecla,name,region,subregion,ne_id,level,geometry
0,Island,NaN,NaN,NaN,NaN,0,"MULTIPOLYGON (((-159.042 -79.708, -159.542 -79..."
1,Coast,NaN,NaN,NaN,NaN,0,"MULTIPOLYGON (((-158.042 -84.792, -163.176 -84..."
2,Range/mtn,NaN,NaN,NaN,NaN,0,"MULTIPOLYGON (((-154.926 -85.316, -150.290 -85..."
3,Pen/cape,NaN,NaN,NaN,NaN,0,"MULTIPOLYGON (((-75.336 -46.936, -75.574 -46.9..."
4,Desert,NaN,NaN,NaN,NaN,0,"MULTIPOLYGON (((140.931 -31.089, 140.805 -31.0..."
...,...,...,...,...,...,...,...
1038,Pen/cape,Coromandel,Oceania,New Zealand,1.159105e+09,1,"POLYGON ((175.336 -36.411, 175.465 -36.418, 17..."
1039,Plateau,N.I. Volcanic Plateau,Oceania,New Zealand,1.159105e+09,1,"POLYGON ((175.721 -38.867, 175.980 -38.956, 17..."
1040,Coast,Marlborough Sounds,Oceania,New Zealand,1.159105e+09,1,"POLYGON ((173.725 -40.798, 173.856 -40.708, 17..."
1041,Pen/cape,Otago Pen.,Oceania,New Zealand,1.159105e+09,1,"POLYGON ((170.654 -45.716, 170.740 -45.721, 17..."


**Add id**

In [28]:
data['id'] = np.arange(len(data))

Save `GeoDataFrame` as `GeoJSON`

In [29]:
data.to_file("../data/mbtiles/ne_10m_geography_regions.json", driver="GeoJSON")

Save `GeoDataFrame` as `Shapefile`

In [30]:
data.to_file("../data/mbtiles/ne_10m_geography_regions/ne_10m_geography_regions.shp")

### Add bounding box

In [31]:
datasets = ['ne_10m_geography_regions', 'ecoregions_by_biome', 'political_boundaries', 'hydrological_basins']
datasets = ['ne_10m_geography_regions']
for dataset in datasets:
    gdf = gpd.read_file(f"../data/mbtiles/{dataset}/{dataset}.shp")
    gdf['bbox'] = gdf['geometry'].apply(lambda x: str(list(x.bounds)))
    gdf.to_file(f"../data/mbtiles/{dataset}.json", driver="GeoJSON")
    gdf.to_file(f"../data/mbtiles/{dataset}/{dataset}.shp")

<a id='create_mbtiles_2'></a>
### Create `MBTiles`

In [32]:
layers = {'Hydrological basins': 'hydrological_basins.json',
         'Ecoregions': 'ecoregions_by_biome.json',
         'Political boundaries': 'political_boundaries.json',
         'Physical geography regions': 'ne_10m_geography_regions.json'}

layers = {'Physical geography regions': 'ne_10m_geography_regions.json'}

for layer_name, file in layers.items():
    print(layer_name)
    source_path = "../data/mbtiles/"+file
    dest_path = "../data/mbtiles/"+file.split('.')[0]+".mbtiles"
    create_mbtiles(source_path, dest_path, layer_name, opts="-zg --drop-densest-as-needed --extend-zooms-if-still-dropping --force --read-parallel")

Physical geography regions
Processing: tippecanoe -o ../data/mbtiles/ne_10m_geography_regions.mbtiles -l Physical geography regions -zg --drop-densest-as-needed --extend-zooms-if-still-dropping --force --read-parallel ../data/mbtiles/ne_10m_geography_regions.json
Task created
Finished processing


<a id='show_mbtiles'></a>
### Show `MBTiles` in our localhost

In [18]:
!mbview --port 9000 ../data/mbtiles/ne_10m_geography_regions.mbtiles

Listening on http://localhost:9000
^C


<a id='serve_mbtiles'></a>
### 7. Serve `MBTiles` in our localhost

In [ ]:
!docker run --rm -p 8080:8000 -v /Users/ikersanchez/Vizzuality/GitHub/sci_team_data_bank/Projects/soils-revealed/data/mbtiles:/tilesets  consbio/mbtileserver

Paste in your browser the following:

`http://localhost:8080/services/hydrological_basins`

And to see the tiles on a map:

`http://localhost:8080/services/hydrological_basins/map`